In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import time
import os
from io import BytesIO
from tqdm import tqdm


def add_username_watermark (username = "shriansh.sahu" , color = 'gray' , alpha = 0.7):

    plt.gca().text(
        0.98,0.02,username,
        ha = 'right' , va = 'bottom',
        transform = plt.gca().transAxes,
        fontsize = 10 , color = 'gray' , alpha = 0.7
    )


class Identity:
    def forward(self,z) : return z
    def backward(self,z): return np.ones(z.shape)

class Sigmoid:
    def forward(self,z) : return 1/(1 + np.exp(-np.clip(z, -500 , 500)))
    def backward(self,z):
        s = self.forward(z)
        return s * (1-s)

class Tanh:
    def forward(self,z) : return np.tanh(z)
    def backward(self,z): return 1 - np.tanh(z) ** 2

class ReLU:
    def forward(self,z) : return np.maximum(0,z)
    def backward(self,z): return (z >0).astype(float)

class MSE:
    def forward(self, y_true , y_pred):
        return np.mean((y_pred - y_true)**2)

    def backward(self, y_true , y_pred):
        return 2*(y_pred - y_true) / y_true.size

class BCE:
    def forward(self, y_true , y_pred):
        epsilon = 1e-5
        y_pred = np.clip(y_pred , epsilon , 1-epsilon)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def backward(self, y_true , y_pred):
       epsilon = 1e-5
       y_pred = np.clip(y_pred , epsilon , 1-epsilon)
       return ((1 - y_true) / ( 1 - y_pred) - y_true / y_pred) / y_true.size


class Linear:

    def __init__(self ,input_width  , output_width , activation ):
        limit = np.sqrt(6 / (input_width + output_width))
        self.weights = np.random.uniform(-limit , limit, ( input_width, output_width))
        self.biases = np.zeros((1 , output_width))
        self.activation = activation()
        self.input , self.z , self.a = None, None, None
        self.grad_weights_cumulative = np.zeros_like(self.weights)
        self.grad_biases_cumulative = np.zeros_like(self.biases)

    def forward(self, input_data):
        self.input , self.z = input_data , np.dot(input_data , self.weights) + self.biases
        self.a = self.activation.forward(self.z)
        return self.a

    def backward( self , upstream_gradient):
        d_z = upstream_gradient * self.activation.backward(self.z)
        self.grad_weights_cumulative += np.dot(self.input.T , d_z)
        self.grad_biases_cumulative += np.sum(d_z , axis = 0 , keepdims= True)
        return np.dot(d_z , self.weights.T)


class Model:
    def __init__(self, layers , loss_function):
        self.layers , self.loss_fn = layers, MSE()

    def forward(self, x):
            for layer in self.layers : x = layer.forward(x)
            return x

    def backward(self, y_true , y_pred):
            grad = self.loss_fn.backward(y_true , y_pred)
            for layer in reversed(self.layers): grad = layer.backward(grad)

    def train(self,x,y):
            self.zero_grad()
            y_pred = self.forward(x)
            loss = self.loss_fn.forward(y , y_pred)
            self.backward(y , y_pred)
            return loss , y_pred

    def zero_grad(self):
            for layer in self.layers:
                layer.grad_weights_cumulative.fill(0)
                layer.grad_biases_cumulative.fill(0)

    def update(self, learning_rate):
            for layer in self.layers:
                layer.weights -= learning_rate * layer.grad_weights_cumulative
                layer.biases -= learning_rate * layer.grad_biases_cumulative


def get_raw(coords):
  return coords

def get_polynomial(coords , order = 5):

  x,y = coords[: , 0:1] , coords[: , 1:2]
  features = []
  for i in range(1 , order +1):
    for j in range(i+1)
      features.append((x ** (i-j)) * (y ** j))
  return np.hstack([coords] ,features)


def get_fourier(coords , freq = 5):
  x,y = coords[: , 0:1] , coords[: , 1:2]
  features = []
  for i in range(1 , freq +1):
    features.append(np.sin(2 * np.pi * i * x))
    features.append(np.cos(2 * np.pi * i * x))
    features.append(np.sin(2 * np.pi * i * y))
    features.append(np.cos(2 * np.pi * i * y))
  return np.hstack(features)


def Modular_Dataloader(img_path , image_type , order = None ,freq = None , size = (256,256)):
  img = Image.open(img_path)
  img = img.resize(size)
  img_np = np.array(img.convert('L' if image_type == 'gray' else 'RGB')) /255.0
  h ,w = img_np.shape[:2]
  y_coords , x_coords = np.mgrid[0:h, 0:w]
  coords = np.stack((x_coords.ravel()/ (w-1) , y_coords.ravel() /(h-1)) , axis = 1)
  if method == "Raw" : features = get_raw(coords)
  elif method == "Polynomial" :
     features = get_polynomial(coords , order)
     features = (features - features.min(axis = 0)) / (features.max(axis = 0) - features.min(axis = 0) + 1e-8)
  elif method == "Fourier" : features = get_fourier(coords , freq)
  else: raise ValueError("Invalid method")
  target_pixels = img_np.reshape(-1,1 if image_type == "Gray" else 3)
  return features , target_pixels ,(h,w)

def train_with__model(imag_path , image_type ,  order = None ,freq = None , mlp_hidden_layers = [64,128,128] , lr = 1e-3 , use_early_stopping = False):

    file_stem = os.path.basename(img_path).split(".")[0]
    param_str = f"o{order}" if order is not None else (f"f{freq}" if freq is not None else "")
    method_str = f"{method.lower()}_{param_str}" if param_str else method.lower()
    output_dir = os.path.join("epochs_outputs" , file_stem , method_str)
    os.makedirs(output_dir , exist_ok = True)

    print(f" Training Method = {method} , Param = {param_str or '_'} , Image = {file_stem}")
    print(f"Saving epoch outputs to : '{output_dir}")

    features , target, shape = Modular_Dataloader(img_path , image_type , order , freq)
    h ,w = shape

    input_width = features.shape[1]
    output_width = target.shape[1]

    layers_dims = [input_dim] + mlp_hidden_layers + [output_dim]
    layers = [Linear(layer_dims[i] , layers_dims[i+1] , ReLU) for i in range (len(layers_dims) - 2)]
    layers.append(Linear(layers_dims[-2] , layers_dims[-1] , Sigmoid))
    model = Model(layers)

    losses = []
    reconstructed_images = []
    start_time = time.time()


    best_loss ,patience_counter, patience , min_delta = float('inf') ,0 ,5 ,1e-6

    pbar = tqdm(range(epochs) , desc = "Training")
    for epoch in pbar:
      loss , predictions = model.train(features , target)
      model.update(learning_rate = lr)
      losses.append(loss)
      pbar.set_description(f"Loss: {loss:.6f}")


      if use_early_stopping:
        if loss < best_loss - min_delta:
          best_loss = loss
          patience_counter = 0
        else:
          patience_counter += 1
        if patience_counter >= patience:
          print(f"\nEarly stopping at epoch {epoch+1} as loss did not improve.")
          break

      img_pred = (predictions.reshape(shape , output_dim) * 255).astype(np.uint8)
      current_image = Image.fromarray(img_pred)

      draw = ImageDraw.Draw(current_image)
      text_position = (w-10, h -10)
      draw.text(text_position , username , font= font , fill= (128,128,128,150) , anchor = "rs")


      current_image.save(os.path.join(output_dir , f"epoch_{epoch+1:03d}.png"))
      reconstructed_images.append(current_image)


     epoch_time = (time.time() - start_time) / (epoch+1)
     return{"method" : method , "final_loss": losses[-1] , "epoch_time": epoch_time ,"imput_params": imput_dim ,"losses": losses , "images": reconstructed_images , "param_val" : order if order is not None else freq , "image": os.path.basename(img_path)}


def create_image_gif(raw_res , poly_res , fourier_res , filename):

  print(f"Creating image-only GIF: {filename}...")
  frames = []
  epochs = len(raw_res["images"])

  for i in range(epochs):

    if i % 2 != 0:
      continue

    fig ,axes = plt.subplots(1,3 , figsize = (15,5.5))
    fig.suptitle(f"Image Reconstruction - Epoch{i+1}" , fontsize = 16)

    results_list = [raw_res , poly_res , fourier_res]
    titles = ["Raw" , "Polynomial" , "Fourier"]

    for ax , res , title in zip(axes , results_list , titles):
      ax.imshow(res["images"][i] , cmap = 'gray' if len(res['images'][i].mode) == 1 else None)
      ax.set_title(f"{title} \n Loss : {res['losses'][i] : .5f}")
      ax.set_xticks([])
      ax.set_yticks([])
      ax.text(0.98,0.02, username , ha= 'right' , va  = 'bottom' , transform = ax.transAxes , fontsize = 10 ,color = 'white', alpha = 0.7)


    plt.tight_layout(rect = [0,0,1,0.95])


    buf = BytesIO()
    plt.savefig(buf , format = 'png')
    buf.seek(0)
    img = Image.open(buf)
    frames.append(img)
    plt.close(fig)

  frames[0].save(filename, save_all=True, append_images=frames[1:], duration=150, loop=0)
  print(f"GIF saved as {filename}")


def save_loss_curves(raw_res , poly_res , fourier_res , filename):

  print(f"Saving loss curves to : {filename}")

  results_map = {"raw": raw_res , "polynomial":poly_res ,"fourier" fourier_res}

  for name , res in results_map.items():
    plt.figure(figsize = (10,6))
    epochs = len(res['losses'])

    plt.plot(range(epochs) ,res['losses'] , label = 'Training Loss')

    plt.title(f"Loss Curve - {name.capatalize()}({file_stem})")
    plt.xlabel('Epochs')
    plt.ylabel('MSE_Loss')
    plt.grid(True)
    plt.legend()

    add_username_watermark()
    plt.savefig(f"loss_curve_{name}_{file_stem}.png")
    plt.close()

    print(f"Saved 3 Loss curve plots")


if __name__ == "__main__":

  os.makedirs("epochs_outputs" , exist_ok = True)
   # Part 2.4
  all_results = []
  image_config = [
      {"path" : "/content/smiley.png" , "type" : "Gray" , "epochs" : 50},
      {"path" : "/content/cat.png" , "type" : "RGB" , "epochs" : 150}
  ]

  cat_setting = {
      "path" : "cat.jpg" , "type" : "RGB" , "epochs" :150,
      "arch": [256 ,256 , 256],
      "lr" : 1e-4

  }

  smile_setting = {
      "path" : "smile.jpg" , "type" : "RGB" , "epochs" :150,
      "arch": [256 ,256 , 256],
      "lr" : 1e-4
  }

  image_configs = [cat_setting , smile_setting]

  for img_config in image_configs:
    path = img_config["path"]
    image_type = img_config["type"]
    epochs = img_config["epochs"]

    raw_res = train_with__model(path , image_type ,"Raw" , epochs , mlp_hidden_layers = arch , lr = lr)
    poly_res = [train_with__model(path , image_type ,"Polynomial" , epochs , order = o , mlp_hidden_layers = arch , lr = lr) for o in [5,15,25]]
    fourier_res =[train_with__model(path , image_type ,"Fourier" , epochs , freq = f , mlp_hidden_layers = arch , lr = lr) for f in [5,15,25]]


    all_results.extend([raw_res] + poly_res + fourier_res)
    best_poly = min(poly_res , key = lambda x: x["final_loss"])
    best_fourier = min(fourier_res , key = lambda x: x["final_loss"])

    base_name = os.path.basename(path)
    file_stem = base_name_split('.')[0]

    gif_filename = f"reconstruction_{file_stem}.gif"
    create_image_gif(raw_res , best_poly , best_fourier , gif_filename)

    save_loss_curves(raw_res , best_poly , best_fourier , file_stem)

  print("\n--- Final Results Table (Part 2.4) ---")
  print(f"{'Image':<12} {'Method':<12} {'Param':<8} {'Input Dims':<12} {'Epoch Time (s)':<18} {'Final Loss':<15}")
  print("-" * 80)
  for res in all_results_2_4:
        method, param_val = res['method'], res['param_val']
        param_str = f"k={param_val}" if param_val else '-'
        print(f"{res['img_name']:<12} {method:<12} {param_str:<8} {res['input_params']:<12} {res['epoch_time']:.4f}{'s':<15} {res['final_loss']:.6f}")

  #  Part 2.5
  blur_folder = "blurred"

  if not os.path.exists(blur_folder):
        print(f"ERROR: Folder '{blur_folder}' not found. Please create it and add the blurred images.")
        print("Skipping Part 2.5.")
  else:
        base_losses, fourier_losses, blur_levels = [], [], range(10)

        for blur in blur_levels:
            img_path = os.path.join(blur_folder, f"blur_{blur}.png")
            if not os.path.exists(img_path):
                print(f"Warning: '{img_path}' not found. Skipping this blur level.")
                continue

            base_res = train_with_your_model(img_path, "RGB", "Raw", epochs=100, use_early_stopping=True)
            base_losses.append(base_res['final_loss'])

            fourier_res = train_with_your_model(img_path, "RGB", "Fourier", epochs=100, freq=5, use_early_stopping=True)
            fourier_losses.append(fourier_res['final_loss'])


        if not base_losses or not fourier_losses:
            print("\n Error: Could not generate blur analysis plot because no training was performed.")
        else:
            plt.figure(figsize=(14, 6))
            plt.suptitle("Reconstruction Loss vs. Image Blur Level", fontsize=16)

            plt.subplot(1, 2, 1)
            plt.plot(blur_levels, base_losses, 'o-', label='BASE (Raw)')
            plt.plot(blur_levels, fourier_losses, 's-', label='FOURIER (k=5)')
            plt.xlabel("Blur Level (σ)"); plt.ylabel("Final Reconstruction Loss (MSE)")
            plt.title("Loss vs. Blur (Linear Scale)"); plt.legend(); plt.grid(True)
            add_username_watermark()

            plt.subplot(1, 2, 2)
            plt.plot(blur_levels, base_losses, 'o-', label='BASE (Raw)')
            plt.plot(blur_levels, fourier_losses, 's-', label='FOURIER (k=5)')
            plt.yscale('log'); plt.xlabel("Blur Level (σ)"); plt.ylabel("Final Reconstruction Loss (MSE)")
            plt.title("Loss vs. Blur (Log Scale)"); plt.legend(); plt.grid(True, which="both", ls="--")
            add_username_watermark()

            plt.tight_layout(rect=[0, 0, 1, 0.95])
            plt.savefig("blur_reconstruction_loss.png")
            print("\n Saved blur analysis plot to 'blur_reconstruction_loss.png'")
            plt.show()













In [ ]:
--- Final Results Table (Part 2.4) ---
Image        Method       Param    Input Dims   Epoch Time (s)     Final Loss
--------------------------------------------------------------------------------
smiley.png   Raw          -        2            0.9894s               0.247236
smiley.png   Polynomial   k=5      22           1.0210s               0.241628
smiley.png   Polynomial   k=15     137          1.1259s               0.225156
smiley.png   Polynomial   k=25     352          1.4515s               0.235171
smiley.png   Fourier      k=5      21           1.0303s               0.235218
smiley.png   Fourier      k=15     61           1.0721s               0.265228
smiley.png   Fourier      k=25     101          1.1219s               0.290214
cat.jpg      Raw          -        2            2.5567s               0.093617
cat.jpg      Polynomial   k=5      22           2.6126s               0.093524
cat.jpg      Polynomial   k=15     137          2.8392s               0.090355
cat.jpg      Polynomial   k=25     352          3.3214s               0.092172
cat.jpg      Fourier      k=5      21           2.6393s               0.105802
cat.jpg      Fourier      k=15     61           2.6839s               0.088262
cat.jpg      Fourier      k=25     101          2.7988s               0.094224

As a image get blurrier , the reconstruction task get easier , but the special advantage of the fourier method disappears. Blurring an image erases its high frequency details sharp edges and textures . The fourier models main strength is its ability to reconstruct these exact high frequency details. The Raw model main weakness is its inability to hnadel them. when you reconstruct a heavily blurred image , there are no high frequency details left to learn . This takes away the fourier models's advantage and hides the Raw model weakness, causing the performance to beocme much more similar